In [1]:
import pandas as pd

In [2]:
filter_columns=pd.read_csv('filtered_dataset/plan_summary_filtered_columns_1.csv')

In [ ]:
prepared_data=pd.read_csv('agg/prepared_data_2.csv')

In [3]:
prepared_data.shape

(178325, 17)

In [ ]:
connected_device=pd.read_csv('dataset/connected_devices.csv')

In [ ]:
gp_id=pd.read_csv('dataset/group_id_files/query_result_2025-05-13T11_13_11.122+06_30.csv')

In [7]:
gp_id.shape

(592217, 6)

In [8]:
gp_id.head()

ID  Group Ref ID Create Date  Status        Obsolete Date Group Name
0  330252  NET000330252  2021-08-14   False  2022-03-08T17:06:06        NaN
1  281477  NET000281477  2021-04-20   False  2022-03-08T17:06:06        NaN
2  342751  NET000342751  2021-09-09   False  2022-03-08T17:06:06        NaN
3  186204  NET000186204  2020-05-20   False  2022-03-08T17:06:06        NaN
4  404445  NET000404445  2022-02-12   False  2022-03-08T17:06:06        NaN

#### Fetch out only 'Customer ID' from the previous prepared data (178325 Unique Customer IDs)

In [9]:
id_list=pd.DataFrame({'Customer ID':prepared_data['Customer ID'].unique()})

In [10]:
id_list

Customer ID
0           2074729
1           2514023
2           2288502
3           2114773
4           2380314
...             ...
178320      2553933
178321      2017863
178322      2456162
178323      2377702
178324      2206790

[178325 rows x 1 columns]

### Map the unique customer ids 178325 with its corresponding 'Group Ref ID' from Plan Summary.

In [11]:
merge_gp_rf_id=pd.merge(
    id_list,
    filter_columns[['Customer ID','Group Ref ID']],
    on=['Customer ID'],
    how='left'
)

In [12]:
merge_gp_rf_id.shape

(335285, 2)

In [13]:
merge_gp_rf_id.head()

Customer ID                    Group Ref ID
0      2074729  005771922-GROUP-BI0002-6200942
1      2074729  005771922-GROUP-BI0002-6200942
2      2074729  005771922-GROUP-BI0002-6200942
3      2514023                             NaN
4      2514023                    NET000544241

### Map the 'Group Ref ID' with its corresponding 'Group ID' 

In [14]:
merge_gp_id=pd.merge(
    merge_gp_rf_id,
    gp_id[['Group Ref ID','ID']],
    on=['Group Ref ID'],
    how='left'
)

In [15]:
merge_gp_id.shape

(336141, 3)

In [16]:
merge_gp_id.head()

Customer ID                    Group Ref ID        ID
0      2074729  005771922-GROUP-BI0002-6200942   55509.0
1      2074729  005771922-GROUP-BI0002-6200942   55509.0
2      2074729  005771922-GROUP-BI0002-6200942   55509.0
3      2514023                             NaN       NaN
4      2514023                    NET000544241  544245.0

In [17]:
merged_data=merge_gp_id.drop_duplicates()

In [18]:
merged_data.shape

(211735, 3)

In [19]:
merged_data

Customer ID                    Group Ref ID        ID
0           2074729  005771922-GROUP-BI0002-6200942   55509.0
3           2514023                             NaN       NaN
4           2514023                    NET000544241  544245.0
5           2288502                    NET000315985  315985.0
6           2114773                    NET000114548  114548.0
...             ...                             ...       ...
336132      2017863    005033585-GROUP-S0003-142525    1614.0
336134      2456162                    NET000484238  484238.0
336136      2377702                    NET000405568  405568.0
336138      2206790                    NET000221968  221968.0
336139      2206790                    NET000294599  294599.0

[211735 rows x 3 columns]

In [20]:
merged_data['Customer ID'].value_counts()

Customer ID
2327648    48
2132516    26
2015428    26
2403975    25
2101661    24
           ..
2377702     1
2449348     1
2407024     1
2440484     1
2072865     1
Name: count, Length: 178325, dtype: int64

In [21]:
merged_data[merged_data['Customer ID']==2101661]

Customer ID  Group Ref ID        ID
83767      2101661  NET000091025   91025.0
83768      2101661  NET000369084  369084.0
83770      2101661  NET000472027  472027.0
83771      2101661  NET000167191  167191.0
83773      2101661  NET000165936  165936.0
83775      2101661  NET000165995  165995.0
83776      2101661  NET000369099  369099.0
83777      2101661  NET000166082  166082.0
83778      2101661  NET000513694  513694.0
83779      2101661  NET000423526  423526.0
83780      2101661  NET000369104  369104.0
83781      2101661  NET000166174  166174.0
83782      2101661  NET000165956  165956.0
83786      2101661  NET000292201  292201.0
83787      2101661  NET000467045  467045.0
83789      2101661  NET000558957  558963.0
83792      2101661  NET000495681  495681.0
83793      2101661  NET000554816  554822.0
83794      2101661  NET000578486  578494.0
83796      2101661  NET000203912  203912.0
83800      2101661  NET000570843  570851.0
83801      2101661  NET000587888  587897.0
83802      2101661  NET000568617  568624.0
83804      2101661  NET000527696  527698.0

#### A 'Customer ID' having multiple 'Group Ref ID' means that he is using or had used multiple CPE at once

In [22]:
merged_data[merged_data['ID'].isna()]

Customer ID Group Ref ID  ID
3           2514023          NaN NaN
32          2341148          NaN NaN
35          2000532          NaN NaN
48          2324862          NaN NaN
63          2382297          NaN NaN
...             ...          ...  ..
336081      2521204          NaN NaN
336099      2118887          NaN NaN
336106      2246039          NaN NaN
336120      2104005          NaN NaN
336130      2254320          NaN NaN

[26740 rows x 3 columns]

In [23]:
merged_data = merged_data.rename(columns={
    'ID': 'Group ID'
})


### Split data into 'have group id' and 'do not have group id' because cpe history of those who do not have group id cannot be traced back.

In [24]:
have_gp_id=merged_data[merged_data['Group ID'].notna()]

In [25]:
no_gp_id=merged_data[merged_data['Group ID'].isna()]

In [26]:
have_gp_id.shape

(184995, 3)

In [27]:
no_gp_id.shape

(26740, 3)

In [28]:
no_gp_id=no_gp_id[~(no_gp_id['Customer ID'].isin(have_gp_id['Customer ID']))]

In [29]:
no_gp_id.shape

(373, 3)

In [30]:
have_gp_id.head()

Customer ID                    Group Ref ID  Group ID
0      2074729  005771922-GROUP-BI0002-6200942   55509.0
4      2514023                    NET000544241  544245.0
5      2288502                    NET000315985  315985.0
6      2114773                    NET000114548  114548.0
7      2380314                    NET000408095  408095.0

In [31]:
no_gp_id.head()

Customer ID Group Ref ID  Group ID
133      2258145          NaN       NaN
389      2386186          NaN       NaN
505      2225598          NaN       NaN
788      2032089          NaN       NaN
891      2137315          NaN       NaN

In [32]:
have_gp_id.to_csv('connected_device/IdList_with_GpID.csv')

In [33]:
no_gp_id.to_csv('connected_device/IdList_with_no_GpID.csv')

In [ ]:
cpe_history_1=pd.read_csv('dataset/group_id_files/true_status.csv')

In [ ]:
cpe_history_2=pd.read_csv('dataset/group_id_files/false_status.csv')

In [36]:
cpe_history=pd.concat([cpe_history_1,cpe_history_2])

In [37]:
cpe_history.shape

(1538772, 7)

In [38]:
cpe_history

ID  Group ID   Username  Is Owner Create Date  Status  \
0        835598    308640  982511253      True  2021-06-04    True   
1        835599    308640   CJ072767     False  2021-06-04    True   
2        835608    308644  753927717      True  2021-06-04    True   
3        835618    308648   CJ076829     False  2021-06-04    True   
4        835626    308651   CJ074819     False  2021-06-04    True   
...         ...       ...        ...       ...         ...     ...   
970548  1166939    419560  FFA009398     False  2022-04-08   False   
970549  1174732    262790   CJ094921     False  2022-04-25   False   
970550  1177893    258230  FFG055522     False  2022-05-01   False   
970551  1182970    424236  FFA012042     False  2022-05-08   False   
970552  1183216    141432  FFA005593     False  2022-05-09   False   

                Update Date  
0       2022-10-24T05:56:18  
1       2022-10-24T05:56:18  
2       2022-10-24T05:56:18  
3       2022-10-24T05:56:18  
4       2022-10-24T05:56:18  
...                     ...  
970548  2022-05-09T11:02:22  
970549  2022-05-09T11:02:22  
970550  2022-05-09T11:02:22  
970551  2022-05-09T11:02:22  
970552  2022-05-09T11:02:22  

[1538772 rows x 7 columns]

### Remove those Username whose values are not CPE ID

In [39]:
import re

def filter_device_ids(username):
    if isinstance(username, str):
        ids = username.split(',')
        valid_ids = [device for device in ids if re.fullmatch(r'[A-Z]{2,3}\d{6}', device)]
        return ','.join(valid_ids)
    else:
        return ''

cpe_history['Username'] = cpe_history['Username'].apply(filter_device_ids)

In [40]:
cpe_history=cpe_history[cpe_history['Username']!='']

In [41]:
cpe_history

ID  Group ID   Username  Is Owner Create Date  Status  \
1        835599    308640   CJ072767     False  2021-06-04    True   
3        835618    308648   CJ076829     False  2021-06-04    True   
4        835626    308651   CJ074819     False  2021-06-04    True   
8        835672    273471  LCF017391     False  2021-06-04    True   
9        835675    308671  SCP008153     False  2021-06-04    True   
...         ...       ...        ...       ...         ...     ...   
970548  1166939    419560  FFA009398     False  2022-04-08   False   
970549  1174732    262790   CJ094921     False  2022-04-25   False   
970550  1177893    258230  FFG055522     False  2022-05-01   False   
970551  1182970    424236  FFA012042     False  2022-05-08   False   
970552  1183216    141432  FFA005593     False  2022-05-09   False   

                Update Date  
1       2022-10-24T05:56:18  
3       2022-10-24T05:56:18  
4       2022-10-24T05:56:18  
8       2022-10-24T05:56:18  
9       2022-10-24T05:56:18  
...                     ...  
970548  2022-05-09T11:02:22  
970549  2022-05-09T11:02:22  
970550  2022-05-09T11:02:22  
970551  2022-05-09T11:02:22  
970552  2022-05-09T11:02:22  

[703523 rows x 7 columns]

### For Status == False, we will take only the data whose 'Day Gap' between 'Created Date' and 'Update Date' is greater than 4 days. Because those data whose day gap less than 4 days are those CPE test using and not Ok. So change with another CPE again.

In [42]:
cpe_history['Status'].value_counts()

Status
False    427246
True     276277
Name: count, dtype: int64

In [43]:
cpe_history_true=cpe_history[cpe_history['Status']==True]

In [44]:
cpe_history_false=cpe_history[cpe_history['Status']==False]

In [45]:
# Convert to datetime
cpe_history_false['Create Date'] = pd.to_datetime(cpe_history_false['Create Date'])
cpe_history_false['Update Date'] = pd.to_datetime(cpe_history_false['Update Date'])

# Compute day difference
cpe_history_false['Day Gap'] = (cpe_history_false['Update Date'] - cpe_history_false['Create Date']).dt.days

# Filter rows where the day gap is 3 or more
filtered_df = cpe_history_false[cpe_history_false['Day Gap'] >= 4].drop(columns='Day Gap')

# Show the result
print(filtered_df)

             ID  Group ID   Username  Is Owner Create Date  Status  \
4          8899      3442   CG148675     False  2018-09-01   False   
8         15907      6117   CG265109     False  2018-09-01   False   
9         16217      6240   CG131833     False  2018-09-01   False   
11        18684      7204   CG227615     False  2018-09-01   False   
12        20727      7995   CG166637     False  2018-09-01   False   
...         ...       ...        ...       ...         ...     ...   
970540  1102755    402168   CJ096261     False  2022-02-06   False   
970541  1137264     11430  LCF038070     False  2022-03-08   False   
970548  1166939    419560  FFA009398     False  2022-04-08   False   
970549  1174732    262790   CJ094921     False  2022-04-25   False   
970550  1177893    258230  FFG055522     False  2022-05-01   False   

               Update Date  
4      2021-01-16 13:40:57  
8      2021-01-16 13:40:57  
9      2021-01-16 13:40:57  
11     2021-01-16 13:40:57  
12     2021-01

/tmp/ipykernel_39159/2391904853.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cpe_history_false['Create Date'] = pd.to_datetime(cpe_history_false['Create Date'])
/tmp/ipykernel_39159/2391904853.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cpe_history_false['Update Date'] = pd.to_datetime(cpe_history_false['Update Date'])
/tmp/ipykernel_39159/2391904853.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = valu

In [46]:
filtered_df

ID  Group ID   Username  Is Owner Create Date  Status  \
4          8899      3442   CG148675     False  2018-09-01   False   
8         15907      6117   CG265109     False  2018-09-01   False   
9         16217      6240   CG131833     False  2018-09-01   False   
11        18684      7204   CG227615     False  2018-09-01   False   
12        20727      7995   CG166637     False  2018-09-01   False   
...         ...       ...        ...       ...         ...     ...   
970540  1102755    402168   CJ096261     False  2022-02-06   False   
970541  1137264     11430  LCF038070     False  2022-03-08   False   
970548  1166939    419560  FFA009398     False  2022-04-08   False   
970549  1174732    262790   CJ094921     False  2022-04-25   False   
970550  1177893    258230  FFG055522     False  2022-05-01   False   

               Update Date  
4      2021-01-16 13:40:57  
8      2021-01-16 13:40:57  
9      2021-01-16 13:40:57  
11     2021-01-16 13:40:57  
12     2021-01-16 13:40:57  
...                    ...  
970540 2022-05-09 11:02:22  
970541 2022-05-09 11:02:22  
970548 2022-05-09 11:02:22  
970549 2022-05-09 11:02:22  
970550 2022-05-09 11:02:22  

[417930 rows x 7 columns]

In [47]:
cpe_history[cpe_history['Group ID']==330806]

ID  Group ID  Username  Is Owner Create Date  Status  \
412046  895594    330806  CJ081580     False  2021-08-15    True   

                Update Date  
412046  2022-10-24T05:56:18

In [48]:
filtered_cpe_history=pd.concat([filtered_df,cpe_history_true])

In [49]:
import pandas as pd
from datetime import datetime
import calendar

# Convert relevant columns to datetime
filtered_cpe_history['Create Date'] = pd.to_datetime(filtered_cpe_history['Create Date'])
filtered_cpe_history['Update Date'] = pd.to_datetime(filtered_cpe_history['Update Date'], errors='coerce')

# Define the months to process
month_order = ['October', 'November', 'December', 'January', 'February', 'March']
month_start_dates = {
    'October': datetime(2024, 10, 1),
    'November': datetime(2024, 11, 1),
    'December': datetime(2024, 12, 1),
    'January': datetime(2025, 1, 1),
    'February': datetime(2025, 2, 1),
    'March': datetime(2025, 3, 1)
}

device_rows = []

for _, row in filtered_cpe_history.iterrows():
    status = str(row['Status']).strip()  # Ensure string and strip spaces
    start_date = row['Create Date']
    end_date = row['Update Date'] if pd.notnull(row['Update Date']) else datetime(2025, 3, 31)
    group_id = row['Group ID']
    device = row['Username']

    filled = {'Group ID': group_id}

    for month in month_order:
        month_date = month_start_dates[month]

        # Default to empty
        filled[month] = ''

        if status == 'False':
            for month in month_order:
                month_start = month_start_dates[month]
                month_end = datetime(month_start.year, month_start.month, calendar.monthrange(month_start.year, month_start.month)[1])
                
                # Fill if there is any overlap between [start_date, end_date] and [month_start, month_end]
                if start_date <= month_end and end_date >= month_start:
                    filled[month] = device
                else:
                    filled[month] = ''

        elif status == 'True':
            # If Plan Start Date is before October, start from October
            if start_date < datetime(2024, 10, 1):
                filled[month] = device
            else:
                if month_date >= datetime(start_date.year, start_date.month, 1):
                    filled[month] = device

    device_rows.append(filled)

# Create final DataFrame
device_count = pd.DataFrame(device_rows)

# Ensure all expected columns exist
device_count = device_count[['Group ID'] + month_order]

print(device_count)


        Group ID    October   November   December    January   February  \
0           3442                                                          
1           6117                                                          
2           6240                                                          
3           7204                                                          
4           7995                                                          
...          ...        ...        ...        ...        ...        ...   
694202    195061  FFA012839  FFA012839  FFA012839  FFA012839  FFA012839   
694203     45895  FFG055558  FFG055558  FFG055558  FFG055558  FFG055558   
694204    244857  FFA012204  FFA012204  FFA012204  FFA012204  FFA012204   
694205     74714  FFG006465  FFG006465  FFG006465  FFG006465  FFG006465   
694206        87   CF196343   CF196343   CF196343   CF196343   CF196343   

            March  
0                  
1                  
2                  
3                  

In [50]:
device_count.to_csv('connected_device/cpe_history_by_group_id.csv')

In [51]:
# Ensure cpe_id is string
connected_device['cpe_id'] = connected_device['cpe_id'].astype(str)

# Convert record_month to string format 'YYYY-MM'
connected_device['record_month'] = pd.to_datetime(connected_device['record_month']).dt.strftime('%Y-%m')

# Define month mapping for the count columns (edit as needed)
month_map = {
    'October': '2024-10',
    'November': '2024-11',
    'December': '2024-12',
    'January': '2025-01',
    'February': '2025-02',
    'March': '2025-03'
}

# Create a mapping: (cpe_id, record_month) => unique_device_count
device_count_map = connected_device.set_index(['cpe_id', 'record_month'])['unique_device_count'].to_dict()

# Function to compute count from device_count cell
def compute_count(cell, month_name):
    if pd.isna(cell) or cell == '':
        return 0
    cpes = [c.strip() for c in str(cell).split(',')]
    target_month = month_map[month_name]
    total = 0
    for cpe in cpes:
        count = device_count_map.get((cpe, target_month), 0)
        # Debug print for testing
        # print(f'Checking CPE {cpe} in month {target_month}: count = {count}')
        total += count
    return total

# Process each month column
for month in month_map:
    device_count[f'{month} Count'] = device_count[month].apply(lambda x: compute_count(x, month))

/tmp/ipykernel_39159/3362430451.py:5: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  connected_device['record_month'] = pd.to_datetime(connected_device['record_month']).dt.strftime('%Y-%m')


In [52]:
device_count

Group ID    October   November   December    January   February  \
0           3442                                                          
1           6117                                                          
2           6240                                                          
3           7204                                                          
4           7995                                                          
...          ...        ...        ...        ...        ...        ...   
694202    195061  FFA012839  FFA012839  FFA012839  FFA012839  FFA012839   
694203     45895  FFG055558  FFG055558  FFG055558  FFG055558  FFG055558   
694204    244857  FFA012204  FFA012204  FFA012204  FFA012204  FFA012204   
694205     74714  FFG006465  FFG006465  FFG006465  FFG006465  FFG006465   
694206        87   CF196343   CF196343   CF196343   CF196343   CF196343   

            March  October Count  November Count  December Count  \
0                              0               0               0   
1                              0               0               0   
2                              0               0               0   
3                              0               0               0   
4                              0               0               0   
...           ...            ...             ...             ...   
694202  FFA012839             22              28              34   
694203  FFG055558              8               9              11   
694204  FFA012204             14              14              22   
694205  FFG006465              5               5               5   
694206   CF196343             11              12              11   

        January Count  February Count  March Count  
0                   0               0            0  
1                   0               0            0  
2                   0               0            0  
3                   0               0            0  
4                   0               0            0  
...               ...             ...          ...  
694202             40              40           55  
694203             10               9            8  
694204             20              15           16  
694205              5               5            5  
694206             11              12           15  

[694207 rows x 13 columns]

In [53]:
device_count.to_csv('connected_device/cpe_history_assigned_device_count.csv')

In [54]:
agg_connected_device=device_count[['Group ID','October Count','November Count','December Count','January Count','February Count','March Count']]

In [55]:
agg_connected_device

Group ID  October Count  November Count  December Count  \
0           3442              0               0               0   
1           6117              0               0               0   
2           6240              0               0               0   
3           7204              0               0               0   
4           7995              0               0               0   
...          ...            ...             ...             ...   
694202    195061             22              28              34   
694203     45895              8               9              11   
694204    244857             14              14              22   
694205     74714              5               5               5   
694206        87             11              12              11   

        January Count  February Count  March Count  
0                   0               0            0  
1                   0               0            0  
2                   0               0            0  
3                   0               0            0  
4                   0               0            0  
...               ...             ...          ...  
694202             40              40           55  
694203             10               9            8  
694204             20              15           16  
694205              5               5            5  
694206             11              12           15  

[694207 rows x 7 columns]

### Calculate Average by Group ID

In [56]:
count_columns = ['October Count','November Count', 'December Count', 'January Count', 
                 'February Count', 'March Count']

# Step 1: Ensure count columns are numeric
agg_connected_device[count_columns] = agg_connected_device[count_columns].apply(pd.to_numeric, errors='coerce')

# Step 2: Replace 0 with NaN to exclude them from mean calculation
masked = agg_connected_device.copy()
masked[count_columns] = masked[count_columns].replace(0, pd.NA)

# Step 3: Group by 'Group ID' and calculate mean ignoring zeros (NaNs now)
grouped_df = masked.groupby('Group ID', as_index=False)[count_columns].mean()

# Step 4: Optional - round for display
grouped_df[count_columns] = grouped_df[count_columns].round(1)


/tmp/ipykernel_39159/3832942030.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  agg_connected_device[count_columns] = agg_connected_device[count_columns].apply(pd.to_numeric, errors='coerce')


In [57]:
grouped_df

Group ID October Count November Count December Count January Count  \
0              4           NaN            7.0            5.0           3.0   
1              6           NaN            NaN            NaN           NaN   
2              7          12.0           12.0           19.0          14.0   
3              8           NaN            NaN            NaN           NaN   
4             10           NaN            NaN            NaN           NaN   
...          ...           ...            ...            ...           ...   
540161    627526           NaN            NaN            NaN           NaN   
540162    627527           NaN            NaN            NaN           NaN   
540163    627528           NaN            NaN            NaN           NaN   
540164    627529           NaN            NaN            NaN           NaN   
540165    627530           NaN            NaN            NaN           NaN   

       February Count March Count  
0                 7.0         NaN  
1                 NaN         NaN  
2                17.0        20.0  
3                 NaN         NaN  
4                 NaN         NaN  
...               ...         ...  
540161            NaN         NaN  
540162            NaN         NaN  
540163            NaN         NaN  
540164            NaN         NaN  
540165            NaN         NaN  

[540166 rows x 7 columns]

### Checking

In [58]:
device_count['Group ID'].value_counts().head(20)

Group ID
527208    10
8988      10
120460     9
302022     9
13018      9
229950     9
166139     9
449016     8
33285      8
190168     8
234600     8
106781     8
31933      8
322857     8
144775     8
485677     8
274059     8
535419     8
102776     8
146560     8
Name: count, dtype: int64

In [59]:
device_count[device_count['Group ID']==603408]

Group ID October November December    January   February      March  \
100422    603408                                       LCF046694              
203117    603408                            LCF030414                         
204177    603408                                       LCF000630  LCF000630   
204644    603408                            LCF021161  LCF021161              
584847    603408                                                              

        October Count  November Count  December Count  January Count  \
100422              0               0               0              0   
203117              0               0               0              8   
204177              0               0               0              0   
204644              0               0               0             11   
584847              0               0               0              0   

        February Count  March Count  
100422              16            0  
203117               0            0  
204177              10           12  
204644               5            0  
584847               0            0

In [60]:
connected_device[connected_device['cpe_id']=='LCF034587']

Unnamed: 0 record_month     cpe_id  unique_device_count
250974       250974      2023-01  LCF034587                   11
519460       519460      2023-02  LCF034587                   13
785780       785780      2023-03  LCF034587                   10
5477325     5477325      2024-10  LCF034587                    5
5640107     5640107      2024-11  LCF034587                    5
5835890     5835890      2024-12  LCF034587                   12
6051422     6051422      2025-01  LCF034587                    9
6266992     6266992      2025-02  LCF034587                   12
6530171     6530171      2025-03  LCF034587                   11
6799332     6799332      2025-04  LCF034587                   20
7130757     7130757      2022-07  LCF034587                   10
7642422     7642422      2022-09  LCF034587                   10
7901861     7901861      2022-10  LCF034587                   11
8163944     8163944      2022-11  LCF034587                   11
8428719     8428719      2022-12  LCF034587                   13

In [61]:
connected_device[connected_device['cpe_id']=='ACL000295']

Empty DataFrame
Columns: [Unnamed: 0, record_month, cpe_id, unique_device_count]
Index: []

In [62]:
agg_connected_device[agg_connected_device['Group ID']==355446]

Group ID  October Count  November Count  December Count  \
62881     355446              0               0              12   
68661     355446              0               0               0   
122650    355446              0               0              12   
183300    355446              0               0               0   
210678    355446              0               0               0   
218597    355446              0               0               0   
293582    355446              0               0               0   
470903    355446              0               0               0   

        January Count  February Count  March Count  
62881               0               0            0  
68661               9               0            0  
122650              9               0            0  
183300              9              12           11  
210678              0               0           11  
218597              0               0           11  
293582              0               0            0  
470903              0               0            0

In [63]:
grouped_df[grouped_df['Group ID']==355446]

Group ID October Count November Count December Count January Count  \
269496    355446           NaN            NaN           12.0           9.0   

       February Count March Count  
269496           12.0        11.0

### Calculation 'overall_avg_device_count' and 'avg_device_count_last_2_months' 

In [64]:
# Define the month columns in order
month_cols = ['October Count','November Count', 'December Count', 'January Count', 
              'February Count', 'March Count']

# Compute overall average (ignoring NaNs)
grouped_df['overall_avg_device_count'] = grouped_df[month_cols].mean(axis=1, skipna=True)

# Function to compute average of last 1–2 non-NaN values
def avg_last_2(series):
    non_na = series.dropna().values
    if len(non_na) >= 2:
        return (non_na[-1] + non_na[-2]) / 2
    elif len(non_na) == 1:
        return non_na[-1]
    else:
        return None 

# Apply to each row to compute avg of last 2 months
grouped_df['avg_device_count_last_2_months'] = grouped_df[month_cols].apply(avg_last_2, axis=1)

# round the results
grouped_df['overall_avg_device_count'] = grouped_df['overall_avg_device_count'].round(2)
grouped_df['avg_device_count_last_2_months'] = grouped_df['avg_device_count_last_2_months'].round(2)

In [65]:
grouped_df

Group ID October Count November Count December Count January Count  \
0              4           NaN            7.0            5.0           3.0   
1              6           NaN            NaN            NaN           NaN   
2              7          12.0           12.0           19.0          14.0   
3              8           NaN            NaN            NaN           NaN   
4             10           NaN            NaN            NaN           NaN   
...          ...           ...            ...            ...           ...   
540161    627526           NaN            NaN            NaN           NaN   
540162    627527           NaN            NaN            NaN           NaN   
540163    627528           NaN            NaN            NaN           NaN   
540164    627529           NaN            NaN            NaN           NaN   
540165    627530           NaN            NaN            NaN           NaN   

       February Count March Count overall_avg_device_count  \
0                 7.0         NaN                      5.5   
1                 NaN         NaN                      NaN   
2                17.0        20.0                15.666667   
3                 NaN         NaN                      NaN   
4                 NaN         NaN                      NaN   
...               ...         ...                      ...   
540161            NaN         NaN                      NaN   
540162            NaN         NaN                      NaN   
540163            NaN         NaN                      NaN   
540164            NaN         NaN                      NaN   
540165            NaN         NaN                      NaN   

        avg_device_count_last_2_months  
0                                  5.0  
1                                  NaN  
2                                 18.5  
3                                  NaN  
4                                  NaN  
...                                ...  
540161                             NaN  
540162                             NaN  
540163                             NaN  
540164                             NaN  
540165                             NaN  

[540166 rows x 9 columns]

In [66]:
grouped_df[grouped_df['Group ID']==355446]

Group ID October Count November Count December Count January Count  \
269496    355446           NaN            NaN           12.0           9.0   

       February Count March Count overall_avg_device_count  \
269496           12.0        11.0                     11.0   

        avg_device_count_last_2_months  
269496                            11.5

In [67]:
merged_df=have_gp_id.merge(grouped_df, on='Group ID', how='left')

In [68]:
merged_df

Customer ID                    Group Ref ID  Group ID October Count  \
0           2074729  005771922-GROUP-BI0002-6200942   55509.0          10.0   
1           2514023                    NET000544241  544245.0           4.0   
2           2288502                    NET000315985  315985.0           4.0   
3           2114773                    NET000114548  114548.0           7.0   
4           2380314                    NET000408095  408095.0           5.0   
...             ...                             ...       ...           ...   
184990      2017863    005033585-GROUP-S0003-142525    1614.0          14.0   
184991      2456162                    NET000484238  484238.0           4.0   
184992      2377702                    NET000405568  405568.0          12.0   
184993      2206790                    NET000221968  221968.0           NaN   
184994      2206790                    NET000294599  294599.0           4.0   

       November Count December Count January Count February Count March Count  \
0                 8.0           10.0           7.0           12.0        11.0   
1                 5.0            5.0           5.0            5.0         6.0   
2                 3.0            5.0           4.0            5.0         7.0   
3                 6.0           11.0          10.0            8.0         8.0   
4                 5.0            7.0          11.0            9.0         7.0   
...               ...            ...           ...            ...         ...   
184990           13.0           15.0          20.0           13.0        13.0   
184991            3.0           12.0          11.0            6.0         5.0   
184992            9.0           17.0          14.0           17.0        18.0   
184993            NaN            NaN           NaN            NaN         NaN   
184994            3.0            5.0           4.0            5.0         6.0   

       overall_avg_device_count  avg_device_count_last_2_months  
0                      9.666667                            11.5  
1                           5.0                             5.5  
2                      4.666667                             6.0  
3                      8.333333                             8.0  
4                      7.333333                             8.0  
...                         ...                             ...  
184990                14.666667                            13.0  
184991                 6.833333                             5.5  
184992                     14.5                            17.5  
184993                      NaN                             NaN  
184994                      4.5                             5.5  

[184995 rows x 11 columns]

In [69]:
merged_df['Customer ID'].value_counts().head(30)

Customer ID
2327648    44
2015428    26
2132516    25
2403975    24
2101661    24
2482632    21
2172782    20
2261711    17
2404679    16
2411865    14
2140128    13
2442314    11
2206336    10
2435056    10
2552500    10
2414027     9
2340224     9
2473847     8
2467044     8
2407673     8
2184088     7
2525327     7
2126265     7
2489105     7
2516855     7
2571682     7
2525445     6
2259723     6
2181302     6
2543559     6
Name: count, dtype: int64

### below example means the customer 2543559 is using 6 CPE in March

In [70]:
merged_df[merged_df['Customer ID']==2543559]

Customer ID  Group Ref ID  Group ID October Count November Count  \
83721      2543559  NET000574958  574966.0          17.0           15.0   
83722      2543559  NET000612064  612078.0           NaN            NaN   
83723      2543559  NET000595379  595388.0           NaN            NaN   
83724      2543559  NET000606402  606415.0           NaN            NaN   
83725      2543559  NET000612072  612086.0           NaN            NaN   
83726      2543559  NET000597944  597953.0           NaN            NaN   

      December Count January Count February Count March Count  \
83721           36.0          25.0           25.0        19.5   
83722            NaN           NaN            4.0        27.0   
83723           10.0           7.0            8.0        16.0   
83724            NaN           NaN            9.0         9.0   
83725            NaN           NaN            1.0        23.0   
83726            3.0           5.0            4.0         4.5   

      overall_avg_device_count  avg_device_count_last_2_months  
83721                22.916667                           22.25  
83722                     15.5                           15.50  
83723                    10.25                           12.00  
83724                      9.0                            9.00  
83725                     12.0                           12.00  
83726                    4.125                            4.25

In [71]:
merged_df.to_csv('connected_device/can_check_how_many_cpe_per_customer_have_via_CustomerID.csv')

In [72]:
device_count.to_csv('connected_device/can_trace_monthly_used_cpe_name_via_GroupID.csv')

In [73]:
merged_df.head()

Customer ID                    Group Ref ID  Group ID October Count  \
0      2074729  005771922-GROUP-BI0002-6200942   55509.0          10.0   
1      2514023                    NET000544241  544245.0           4.0   
2      2288502                    NET000315985  315985.0           4.0   
3      2114773                    NET000114548  114548.0           7.0   
4      2380314                    NET000408095  408095.0           5.0   

  November Count December Count January Count February Count March Count  \
0            8.0           10.0           7.0           12.0        11.0   
1            5.0            5.0           5.0            5.0         6.0   
2            3.0            5.0           4.0            5.0         7.0   
3            6.0           11.0          10.0            8.0         8.0   
4            5.0            7.0          11.0            9.0         7.0   

  overall_avg_device_count  avg_device_count_last_2_months  
0                 9.666667                            11.5  
1                      5.0                             5.5  
2                 4.666667                             6.0  
3                 8.333333                             8.0  
4                 7.333333                             8.0

In [74]:
filter_merge_df=merged_df[['Group ID','Customer ID','overall_avg_device_count','avg_device_count_last_2_months']]

In [75]:
filter_merge_df.head()

Group ID  Customer ID overall_avg_device_count  \
0   55509.0      2074729                 9.666667   
1  544245.0      2514023                      5.0   
2  315985.0      2288502                 4.666667   
3  114548.0      2114773                 8.333333   
4  408095.0      2380314                 7.333333   

   avg_device_count_last_2_months  
0                            11.5  
1                             5.5  
2                             6.0  
3                             8.0  
4                             8.0

In [76]:
merge_prepared_data=prepared_data.merge(filter_merge_df, on='Customer ID', how='left')

In [77]:
merge_prepared_data

Unnamed: 0.1  Unnamed: 0  Customer ID  Tk Count Within 30 Days  \
0                  0           0      2074729                        0   
1                  1           1      2514023                        0   
2                  2           2      2288502                        0   
3                  3           3      2114773                        0   
4                  4           4      2380314                        0   
...              ...         ...          ...                      ...   
185363        178321      178321      2017863                        1   
185364        178322      178322      2456162                        1   
185365        178323      178323      2377702                        1   
185366        178324      178324      2206790                        1   
185367        178324      178324      2206790                        1   

        Tk Count Within 60 Days  Previous Termination Attempts Last Priority  \
0                             1                              0           Low   
1                             1                              0           Low   
2                             2                              0           Low   
3                             0                              0        Medium   
4                             0                              0           Low   
...                         ...                            ...           ...   
185363                        1                              0           Low   
185364                        1                              0           Low   
185365                        1                              0           Low   
185366                        1                              0           Low   
185367                        1                              0           Low   

        Life Time  Average Life Time  Max Life Time Last Ticket Created Date  \
0        0.052234           1.043654       1.937002               2025-01-30   
1        2.070324           1.606961       2.702963               2025-02-24   
2        1.950382           2.320818       3.020289               2025-01-29   
3        3.317627           2.772269       3.317627               2024-10-31   
4        0.962292           1.591817       2.221343               2024-10-14   
...           ...                ...            ...                      ...   
185363   0.000590           0.000590       0.000590               2025-03-01   
185364   5.242523           5.242523       5.242523               2025-03-01   
185365   1.104734           1.104734       1.104734               2025-03-01   
185366   1.402350           1.402350       1.402350               2025-03-01   
185367   1.402350           1.402350       1.402350               2025-03-01   

       Plan Start Date  Loyalty Duration  On Going Ticket  Status  \
0           2018-10-29           2251.97                0  ACTIVE   
1           2024-02-12            372.54                0  ACTIVE   
2           2021-06-22           1297.96                0  ACTIVE   
3           2019-08-19           1872.54                0  ACTIVE   
4           2022-02-23            950.07                0  ACTIVE   
...                ...               ...              ...     ...   
185363      2017-07-22           2738.83                0  ACTIVE   
185364      2023-01-30            750.00                0  ACTIVE   
185365      2022-02-16           1092.97                0     NaN   
185366      2020-09-10           1609.40                0  ACTIVE   
185367      2020-09-10           1609.40                0  ACTIVE   

        Total Data Usage  Average Data Usage  Group ID  \
0              1468978.0            244829.0   55509.0   
1               739934.0            123322.0  544245.0   
2               483469.0             80578.0  315985.0   
3               464183.0             77363.0  114548.0   
4               868139.0            144689.0  408095.0   
...                 

In [78]:
merge_prepared_data[merge_prepared_data['Customer ID']==2543559]

Unnamed: 0.1  Unnamed: 0  Customer ID  Tk Count Within 30 Days  \
83912         80476       80476      2543559                        6   
83913         80476       80476      2543559                        6   
83914         80476       80476      2543559                        6   
83915         80476       80476      2543559                        6   
83916         80476       80476      2543559                        6   
83917         80476       80476      2543559                        6   

       Tk Count Within 60 Days  Previous Termination Attempts Last Priority  \
83912                        6                              0        Medium   
83913                        6                              0        Medium   
83914                        6                              0        Medium   
83915                        6                              0        Medium   
83916                        6                              0        Medium   
83917                        6                              0        Medium   

       Life Time  Average Life Time  Max Life Time Last Ticket Created Date  \
83912   1.937025           0.991195        1.96235               2025-03-17   
83913   1.937025           0.991195        1.96235               2025-03-17   
83914   1.937025           0.991195        1.96235               2025-03-17   
83915   1.937025           0.991195        1.96235               2025-03-17   
83916   1.937025           0.991195        1.96235               2025-03-17   
83917   1.937025           0.991195        1.96235               2025-03-17   

      Plan Start Date  Loyalty Duration  On Going Ticket  Status  \
83912      2024-08-16            209.92                0  ACTIVE   
83913      2024-08-16            209.92                0  ACTIVE   
83914      2024-08-16            209.92                0  ACTIVE   
83915      2024-08-16            209.92                0  ACTIVE   
83916      2024-08-16            209.92                0  ACTIVE   
83917      2024-08-16            209.92                0  ACTIVE   

       Total Data Usage  Average Data Usage  Group ID  \
83912         4712083.0            785347.0  574966.0   
83913         4712083.0            785347.0  612078.0   
83914         4712083.0            785347.0  595388.0   
83915         4712083.0            785347.0  606415.0   
83916         4712083.0            785347.0  612086.0   
83917         4712083.0            785347.0  597953.0   

      overall_avg_device_count  avg_device_count_last_2_months  
83912                22.916667                           22.25  
83913                     15.5                           15.50  
83914                    10.25                           12.00  
83915                      9.0                            9.00  
83916                     12.0                           12.00  
83917                    4.125                            4.25

### Fetch only 'Customer ID' and device counting to combine the device counting via Customer ID

In [79]:
filter_merge_prepared_data=merge_prepared_data[['Customer ID','overall_avg_device_count','avg_device_count_last_2_months']]

In [80]:
filter_merge_prepared_data

Customer ID overall_avg_device_count  avg_device_count_last_2_months
0           2074729                 9.666667                            11.5
1           2514023                      5.0                             5.5
2           2288502                 4.666667                             6.0
3           2114773                 8.333333                             8.0
4           2380314                 7.333333                             8.0
...             ...                      ...                             ...
185363      2017863                14.666667                            13.0
185364      2456162                 6.833333                             5.5
185365      2377702                     14.5                            17.5
185366      2206790                      NaN                             NaN
185367      2206790                      4.5                             5.5

[185368 rows x 3 columns]

In [81]:
columns = ['overall_avg_device_count','avg_device_count_last_2_months']
sum_by_cid = filter_merge_prepared_data.groupby('Customer ID', as_index=False)[columns].sum()

In [82]:
sum_by_cid

Customer ID overall_avg_device_count  avg_device_count_last_2_months
0           2000006                        0                             0.0
1           2000026                10.666667                            10.0
2           2000027                    14.75                            15.5
3           2000042                14.166667                            17.0
4           2000059                 7.666667                             8.5
...             ...                      ...                             ...
178320      2585613                      3.0                             3.0
178321      2585619                      2.0                             2.0
178322      2585632                        0                             0.0
178323      2585668                      4.0                             4.0
178324      2585693                      4.0                             4.0

[178325 rows x 3 columns]

In [83]:
data=prepared_data.merge(sum_by_cid, on='Customer ID', how='left')

In [84]:
data

Unnamed: 0.1  Unnamed: 0  Customer ID  Tk Count Within 30 Days  \
0                  0           0      2074729                        0   
1                  1           1      2514023                        0   
2                  2           2      2288502                        0   
3                  3           3      2114773                        0   
4                  4           4      2380314                        0   
...              ...         ...          ...                      ...   
178320        178320      178320      2553933                        1   
178321        178321      178321      2017863                        1   
178322        178322      178322      2456162                        1   
178323        178323      178323      2377702                        1   
178324        178324      178324      2206790                        1   

        Tk Count Within 60 Days  Previous Termination Attempts Last Priority  \
0                             1                              0           Low   
1                             1                              0           Low   
2                             2                              0           Low   
3                             0                              0        Medium   
4                             0                              0           Low   
...                         ...                            ...           ...   
178320                        1                              0           Low   
178321                        1                              0           Low   
178322                        1                              0           Low   
178323                        1                              0           Low   
178324                        1                              0           Low   

        Life Time  Average Life Time  Max Life Time Last Ticket Created Date  \
0        0.052234           1.043654       1.937002               2025-01-30   
1        2.070324           1.606961       2.702963               2025-02-24   
2        1.950382           2.320818       3.020289               2025-01-29   
3        3.317627           2.772269       3.317627               2024-10-31   
4        0.962292           1.591817       2.221343               2024-10-14   
...           ...                ...            ...                      ...   
178320   1.032442           1.032442       1.032442               2025-03-01   
178321   0.000590           0.000590       0.000590               2025-03-01   
178322   5.242523           5.242523       5.242523               2025-03-01   
178323   1.104734           1.104734       1.104734               2025-03-01   
178324   1.402350           1.402350       1.402350               2025-03-01   

       Plan Start Date  Loyalty Duration  On Going Ticket  Status  \
0           2018-10-29           2251.97                0  ACTIVE   
1           2024-02-12            372.54                0  ACTIVE   
2           2021-06-22           1297.96                0  ACTIVE   
3           2019-08-19           1872.54                0  ACTIVE   
4           2022-02-23            950.07                0  ACTIVE   
...                ...               ...              ...     ...   
178320      2024-10-20            130.09                0  ACTIVE   
178321      2017-07-22           2738.83                0  ACTIVE   
178322      2023-01-30            750.00                0  ACTIVE   
178323      2022-02-16           1092.97                0     NaN   
178324      2020-09-10           1609.40                0  ACTIVE   

        Total Data Usage  Average Data Usage overall_avg_device_count  \
0              1468978.0            244829.0                 9.666667   
1               739934.0            123322.0                      5.0   
2               483469.0             80578.0                 4.666667   
3               464183.0             77363.0                 8.333333   
4  

In [85]:
# Convert MB to GB
data['Total Data Usage'] = data['Total Data Usage'] / 1024
data['Average Data Usage'] = data['Average Data Usage'] / 1024

In [86]:
data['Total Data Usage'] = data['Total Data Usage'].round(2)
data['Average Data Usage'] = data['Average Data Usage'].round(2)
data['overall_avg_device_count'] = data['overall_avg_device_count'].astype(float).round(2)

In [87]:
data['Average Life Time'] = data['Average Life Time'].round(2)
data['Max Life Time'] = data['Max Life Time'].round(2)

In [88]:
data

Unnamed: 0.1  Unnamed: 0  Customer ID  Tk Count Within 30 Days  \
0                  0           0      2074729                        0   
1                  1           1      2514023                        0   
2                  2           2      2288502                        0   
3                  3           3      2114773                        0   
4                  4           4      2380314                        0   
...              ...         ...          ...                      ...   
178320        178320      178320      2553933                        1   
178321        178321      178321      2017863                        1   
178322        178322      178322      2456162                        1   
178323        178323      178323      2377702                        1   
178324        178324      178324      2206790                        1   

        Tk Count Within 60 Days  Previous Termination Attempts Last Priority  \
0                             1                              0           Low   
1                             1                              0           Low   
2                             2                              0           Low   
3                             0                              0        Medium   
4                             0                              0           Low   
...                         ...                            ...           ...   
178320                        1                              0           Low   
178321                        1                              0           Low   
178322                        1                              0           Low   
178323                        1                              0           Low   
178324                        1                              0           Low   

        Life Time  Average Life Time  Max Life Time Last Ticket Created Date  \
0        0.052234               1.04           1.94               2025-01-30   
1        2.070324               1.61           2.70               2025-02-24   
2        1.950382               2.32           3.02               2025-01-29   
3        3.317627               2.77           3.32               2024-10-31   
4        0.962292               1.59           2.22               2024-10-14   
...           ...                ...            ...                      ...   
178320   1.032442               1.03           1.03               2025-03-01   
178321   0.000590               0.00           0.00               2025-03-01   
178322   5.242523               5.24           5.24               2025-03-01   
178323   1.104734               1.10           1.10               2025-03-01   
178324   1.402350               1.40           1.40               2025-03-01   

       Plan Start Date  Loyalty Duration  On Going Ticket  Status  \
0           2018-10-29           2251.97                0  ACTIVE   
1           2024-02-12            372.54                0  ACTIVE   
2           2021-06-22           1297.96                0  ACTIVE   
3           2019-08-19           1872.54                0  ACTIVE   
4           2022-02-23            950.07                0  ACTIVE   
...                ...               ...              ...     ...   
178320      2024-10-20            130.09                0  ACTIVE   
178321      2017-07-22           2738.83                0  ACTIVE   
178322      2023-01-30            750.00                0  ACTIVE   
178323      2022-02-16           1092.97                0     NaN   
178324      2020-09-10           1609.40                0  ACTIVE   

        Total Data Usage  Average Data Usage  overall_avg_device_count  \
0                1434.55              239.09                      9.67   
1                 722.59              120.43                      5.00   
2                 472.14               78.69                      4.67   
3                 453.30               75.55                      8.33  

In [89]:
data[data['Customer ID']==2543559]

Unnamed: 0.1  Unnamed: 0  Customer ID  Tk Count Within 30 Days  \
80476         80476       80476      2543559                        6   

       Tk Count Within 60 Days  Previous Termination Attempts Last Priority  \
80476                        6                              0        Medium   

       Life Time  Average Life Time  Max Life Time Last Ticket Created Date  \
80476   1.937025               0.99           1.96               2025-03-17   

      Plan Start Date  Loyalty Duration  On Going Ticket  Status  \
80476      2024-08-16            209.92                0  ACTIVE   

       Total Data Usage  Average Data Usage  overall_avg_device_count  \
80476           4601.64              766.94                     73.79   

       avg_device_count_last_2_months  
80476                            75.0

In [90]:
data.drop(columns=['Unnamed: 0.1','Unnamed: 0','Life Time','Last Ticket Created Date','Plan Start Date'], inplace=True)

In [91]:
data

Customer ID  Tk Count Within 30 Days  Tk Count Within 60 Days  \
0           2074729                        0                        1   
1           2514023                        0                        1   
2           2288502                        0                        2   
3           2114773                        0                        0   
4           2380314                        0                        0   
...             ...                      ...                      ...   
178320      2553933                        1                        1   
178321      2017863                        1                        1   
178322      2456162                        1                        1   
178323      2377702                        1                        1   
178324      2206790                        1                        1   

        Previous Termination Attempts Last Priority  Average Life Time  \
0                                   0           Low               1.04   
1                                   0           Low               1.61   
2                                   0           Low               2.32   
3                                   0        Medium               2.77   
4                                   0           Low               1.59   
...                               ...           ...                ...   
178320                              0           Low               1.03   
178321                              0           Low               0.00   
178322                              0           Low               5.24   
178323                              0           Low               1.10   
178324                              0           Low               1.40   

        Max Life Time  Loyalty Duration  On Going Ticket  Status  \
0                1.94           2251.97                0  ACTIVE   
1                2.70            372.54                0  ACTIVE   
2                3.02           1297.96                0  ACTIVE   
3                3.32           1872.54                0  ACTIVE   
4                2.22            950.07                0  ACTIVE   
...               ...               ...              ...     ...   
178320           1.03            130.09                0  ACTIVE   
178321           0.00           2738.83                0  ACTIVE   
178322           5.24            750.00                0  ACTIVE   
178323           1.10           1092.97                0     NaN   
178324           1.40           1609.40                0  ACTIVE   

        Total Data Usage  Average Data Usage  overall_avg_device_count  \
0                1434.55              239.09                      9.67   
1                 722.59              120.43                      5.00   
2                 472.14               78.69                      4.67   
3                 453.30               75.55                      8.33   
4                 847.79              141.30                      7.33   
...                  ...                 ...                       ...   
178320            603.57              100.59                     15.00   
178321           3715.63              619.27                     14.67   
178322           1085.61              180.93                      6.83   
178323           2118.32              353.05                     14.50   
178324           1060.81              176.80                      4.50   

        avg_device_count_last_2_months  
0                                 11.5  
1                                  5.5  
2                                  6.0  
3                                  8.0  
4                                  8.0  
...                                ...  
178320                            16.5  
178321                            13.0  
178322                             5.5  
178323                            17.5  
178324                             5.5  

[178325 rows x 14 columns]

In [92]:
data.to_csv('agg/aggregated_connected_device.csv')